# Heatmap geográfico
El objetivo de esta notebook es ver crear una visualización geográfica de la frecuencia con la que aparecen ciertas ciudades (principalmente de Brasil) en un dataset.

### Estructura:
- [Obtener coordenadas](#Obtener-coordenadas)
- [Creando el heatmap](#Creando-el-heatmap)

In [1]:
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os.path

%matplotlib inline

In [2]:
%run limpieza.ipynb
df = get_clean_df()

## Obtener coordenadas

Para generar un 'heatmap' con las ciudades necesitamos sus coordenadas. Para ello se descargó un set de datos de http://www.geonames.org con las ciudades de Brasil (http://download.geonames.org/export/dump/BR.zip) que contiene el nombre de las ciudades y sus coordenadas.
Este dataset contiene los siguientes atributos:
- geonameid         : integer id of record in geonames database
- name              : name of geographical point (utf8) varchar(200)
- asciiname         : name of geographical point in plain ascii characters, varchar(200)
- alternatenames    : alternatenames, comma separated, ascii names automatically transliterated, convenience attribute from alternatename table, varchar(10000)
- latitude          : latitude in decimal degrees (wgs84)
- longitude         : longitude in decimal degrees (wgs84)
- feature class     : see http://www.geonames.org/export/codes.html, char(1)
- feature code      : see http://www.geonames.org/export/codes.html, varchar(10)
- country code      : ISO-3166 2-letter country code, 2 characters
- cc2               : alternate country codes, comma separated, ISO-3166 2-letter country code, 200 characters
- admin1 code       : fipscode (subject to change to iso code), see exceptions below, see file admin1Codes.txt for display names of this code; varchar(20)
- admin2 code       : code for the second administrative division, a county in the US, see file admin2Codes.txt; varchar(80) 
- admin3 code       : code for third level administrative division, varchar(20)
- admin4 code       : code for fourth level administrative division, varchar(20)
- population        : bigint (8 byte int) 
- elevation         : in meters, integer
- dem               : digital elevation model, srtm3 or gtopo30, average elevation of 3''x3'' (ca 90mx90m) or 30''x30'' (ca 900mx900m) area in meters, integer. srtm processed by cgiar/ciat.
- timezone          : the iana timezone id (see file timeZone.txt) varchar(40)
- modification date : date of last modification in yyyy-MM-dd format

De todas estas nosotros utilizaremos únicamente name, asciiname, population, latitude y longitude.

In [3]:
columnas = ['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature class', 'feature code',
            'country code', 'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation',
            'dem', 'timezone', 'modification date']
df2 = pd.read_csv(os.path.join('datasets', 'BR.txt'), sep='\t', header=None, names=columnas, usecols=['name', 'asciiname', 'feature class','latitude', 'longitude'])

Este set de datos viene con varias entradas por ciudad con distintas ubicaciones dentro de las mismas:

In [4]:
df2[df2['name'] == 'Rio de Janeiro'].head()

,name,asciiname,latitude,longitude,feature class
33210,Rio de Janeiro,Rio de Janeiro,-22.25000,-42.50000,A
33211,Rio de Janeiro,Rio de Janeiro,-22.90642,-43.18223,P
42179,Rio de Janeiro,Rio de Janeiro,-18.06667,-45.01667,H
42180,Rio de Janeiro,Rio de Janeiro,-11.84252,-45.17394,H
75734,Rio de Janeiro,Rio de Janeiro,-22.92008,-43.33069,A


Esto se debe a que ciertos registros se refieren a ciudades, y otros parte del terreno que llevan el mismo nombre. Un ejemplo claro es el siguiente registro:

In [5]:
df2.iloc[[42179]]

,name,asciiname,latitude,longitude,feature class
42179,Rio de Janeiro,Rio de Janeiro,-18.06667,-45.01667,H


Si buscamos esas coordendas encontraremos que se refiere al rio 'Rio de Janeiro'. Es por esto que a su vez utilizaremos el atributo 'feature class' y tomaremos solo aquellas con el valor 'P' (que indica registros de ciudades, pueblos, etc.).

In [6]:
df2 = df2[df2['feature class'] == 'P']

Sin embargo notamos que sigue habiendo varias entradas repetidas:

In [7]:
df2.shape[0] == df2['name'].unique().size

False

De todas formas parece que sus coordenadas no difieren en gran medida. Para quedarnos con una sola tomaremos como criterio la primera que aparezca.

In [8]:
df2 = df2.groupby('name').first().reset_index()

Ahora vamos a preparar el dataframe del que queremos hacer el heatmap

In [9]:
df = df[df['country'] == 'Brazil']
df = pd.DataFrame(df['city'].value_counts())
df.columns=['peso']
df = df[df['peso'] > 0]
df.head()

,peso
São Paulo,11711
Rio de Janeiro,3538
Belo Horizonte,2568
Salvador,2314
Brasília,1530


Luego vamos a encontrar la latitud y longitud de las ciudades en nuestro dataframe.

In [10]:
df3 = df.reset_index().merge(df2, left_on='index', right_on='name', how='left').set_index('index')
df3 = df3.drop(columns=['name', 'asciiname', 'feature class'])

Aunque encontramos algunos registros que no tienen las coordenadas asignadas:

In [11]:
df3[pd.isnull(df3['latitude'])].head()

,peso,latitude,longitude
index,,,
Carapicuiba,296,NaN,NaN
Sao Goncalo,239,NaN,NaN
Jaboatao dos Guararapes,204,NaN,NaN
Santo Antonio de Jesus,160,NaN,NaN
Sumare,154,NaN,NaN


Esto puede ocurrir por los tildes o caracteres especiales del portugués, por lo cual ahora realizaremos una búsqueda por la columna 'asciiname'.

In [12]:
df4 = df.reset_index().merge(df2, left_on='index', right_on='asciiname', how='left').set_index('index')
df4 = df4.drop(columns=['name', 'asciiname', 'feature class'])

Ahora queda unir ambos dataframes y ver cuales ciudades no se encontraron.

In [13]:
df5 = df4.combine_first(df3)

In [14]:
df5[pd.isnull(df5['latitude'])]

,peso,latitude,longitude
index,,,
São Miguel do Oeste,12,NaN,NaN


Acá notamos que es solo un registro!

Buscando manualmente encontramos que en el set de datos con las coordenadas el nombre de esta ciudad es ligeramente distinto: Sao Miguel D'Oeste, y por eso no fue encontrado.
Dado que es uno solo podemos setear sus coordenadas manualmente.

In [15]:
df5.loc['São Miguel do Oeste', 'latitude'] = -26.71868
df5.loc['São Miguel do Oeste', 'longitude'] = -53.5194

## Creando el heatmap


In [68]:
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

Mostramos un mapa con las ciudades donde hay al menos una visita

In [63]:
m = folium.Map(tiles='CartoDB positron', location=[-10.656360, -51.767393], zoom_start=4)
marker_cluster = MarkerCluster().add_to(m)
for reg in df5.itertuples():
    folium.Marker([reg[2], reg[3]], popup=folium.Popup(reg[0], parse_html=True)).add_to(marker_cluster)

In [64]:
m

Heatmap con la cantidad de visitas

In [70]:
heatmap = folium.Map(tiles='Mapbox Bright', location=[-10.656360, -51.767393], zoom_start=4)
heat_data = []
for reg in df5.itertuples():
    heat_data.append([reg[2], reg[3], reg[1]])

HeatMap(heat_data).add_to(heatmap)
heatmap

In [39]:
# for reg in df5.itertuples():
#     folium.CircleMarker([reg[2], reg[3]], radius=1, fill_color="#3db7e4").add_to(m)